In [2]:
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import numpy as np
import matplotlib.cm as cm
import matplotlib as mpl
from workflow.scripts.plotting_tools import create_facet_plot
from workflow.scripts.utils import regrid_global
from scipy.stats import theilslopes

In [3]:
ec_forcing = xr.open_dataset(snakemake.input.forcing_ec)
mpi_forcing = xr.open_dataset(snakemake.input.forcing_mpi)
nor_forcing = xr.open_dataset(snakemake.input.forcing_noresm)
vname = snakemake.wildcards.vName
mask_treshold = snakemake.params.get('mask_treshold', -2)
time_slice = snakemake.params.get('time_slice', None)
reg_grid_params = snakemake.config['regrid_params']

In [4]:

if time_slice:
    t_slice = time_slice
else:
    t_slice = slice(-30,None)
ec_forcing = ec_forcing.isel(year=t_slice)
nor_forcing = nor_forcing.isel(year=t_slice)
mpi_forcing = mpi_forcing.isel(year=t_slice)
nor_regrid = regrid_global(nor_forcing.cf.add_bounds(['lon','lat']),lon=4,
                       lat=3)
mpi_regrid = regrid_global(mpi_forcing.cf.add_bounds(['lon','lat']),lon=4,
                           lat=3)
ec_regrid = regrid_global(ec_forcing.cf.add_bounds(['lon','lat']),lon=4,
                           lat=3)

In [5]:
def normalize_z(da):
    return (da-da.mean(dim='year'))/da.std(dim='year')

def percentage_change(da):
    return (da-da.shift(year=1).dropna(dim='year')/da)*100

norm_mpi = normalize_z(mpi_regrid[vname]).to_dataset()
norm_nor = normalize_z(nor_regrid[vname]).to_dataset()
norm_ec = normalize_z(ec_regrid[vname]).to_dataset()

rel_mpi = percentage_change(mpi_regrid[vname]).to_dataset()
rel_nor = percentage_change(nor_regrid[vname]).to_dataset()
rel_ec = percentage_change(ec_regrid[vname]).to_dataset()
merge_standardized = (norm_mpi[vname]+norm_nor[vname]+norm_ec[vname])/3

merge_rel = (rel_mpi[vname]+rel_nor[vname]+rel_ec[vname])/3

In [6]:
def sen_slope(da):
    nx = da.shape[1]
    ny = da.shape[2]
    
    out_da = xr.zeros_like(da.isel(year=0).squeeze())
    intercept = xr.zeros_like(out_da)
    low_bound = xr.zeros_like(out_da)
    high_bound = xr.zeros_like(out_da)
    for i in range(nx):
        for j in range(ny):
            out_da[i,j], intercept[i,j], low_bound[i,j], high_bound[i,j] = theilslopes(da.data[:,i,j])
    out_da.attrs['units'] = 'W m-2 year-1'
    out_da.attrs['long_name'] =  f'Trend of {out_da.attrs.get("long_name", da.name)}'
    out_da = out_da.to_dataset(name=f'Trend_{da.name}')
    out_da['lb'] = low_bound
    out_da['lb'].attrs['long_name'] = "Lower bound of CI"
    out_da['ub'] = high_bound
    out_da['ub'].attrs['long_name'] = "Upper bound of CI"
    out_da['intercept'] = intercept
    out_da['intercept'].attrs['long_name'] = "Intercept of trend"
    return out_da

In [7]:
def plot_absolute_trends():
    if vname == 'DirectEff':
        vmin = -0.15
        vmax = 0.15
    elif vname == 'CloudEff':
        vmin = -0.6
        vmax = 0.6
    else:
        vmin = -0.5
        vmax = 0.5
    nor_trend = sen_slope(nor_regrid[vname])
    ec_trend = sen_slope(ec_regrid[vname])
    mpi_trend = sen_slope(mpi_regrid[vname])

    fig ,ax, cax = create_facet_plot(nplots=9, figsize=(12,6), subplot_kw={'projection':ccrs.Robinson()})
    col1 = ['A', 'D', 'G']
    col2 = ['B', 'E', 'H']
    col3 = ['C', 'F', 'I']
    keys = [f'Trend_{vname}', 'lb', 'ub']
    cmap = cm.get_cmap('PRGn_r', 11)
    titles =[f'Absolute {vname} Trend','Lower bound', 'Upper bound']
    for c1, c2, c3, k, t in zip(col1,col2, col3,keys, titles):

        ec_trend[k].plot(ax=ax[c1], transform = ccrs.PlateCarree(), vmin=vmin, vmax=vmax, cmap=cmap, 
                         add_colorbar=False, add_labels=False)
        mpi_trend[k].plot(ax=ax[c2], transform = ccrs.PlateCarree(), vmin=vmin, vmax=vmax, cmap=cmap, add_colorbar=False,
                         add_labels=False)
        nor_trend[k].plot(ax=ax[c3], transform = ccrs.PlateCarree(), vmin=vmin, vmax=vmax, cmap=cmap, add_colorbar=False,
                         add_labels=False)
        ax[c1].set_title(t, fontsize=10)
        ax[c2].set_title(t,fontsize=10)
        ax[c3].set_title(t,fontsize=10)
    for a in ax:
        ax[a].coastlines()

    fig.text(x=0.24,y=0.96,s='EC-Earth3-AerChem', ha='center', va='center', fontsize=16)
    fig.text(x=0.51,y=0.96,s='MPI-ESM-1-2-HAM', ha='center', va='center', fontsize=16)
    fig.text(x=0.786,y=0.96,s='NorESM2-LM', ha='center', va='center', fontsize=16)
    cb = fig.colorbar(cm.ScalarMappable(mpl.colors.Normalize(vmin=vmin,vmax=vmax),cmap=cmap),cax=cax, extend='both')
    cb.set_label(f'Absolute Trend {vname} [year-1]')
    plt.savefig(snakemake.output.plot_abs, dpi=144, bbox_inches='tight')
plot_absolute_trends()

In [28]:
def plot_relative_trends():
    nor_trend = sen_slope(rel_nor[vname])
    ec_trend = sen_slope(rel_ec[vname])
    mpi_trend = sen_slope(rel_mpi[vname])
    rel_forces = sen_slope(merge_rel)

    fig ,ax, cax = create_facet_plot(nplots=12, figsize=(15,6), subplot_kw={'projection':ccrs.Robinson()})
    col1 = ['A', 'E', 'I']
    col2 = ['B', 'F', 'L']
    col3 = ['C', 'G', 'M']
    col4 = ['D', 'H', 'N']
    keys = [f'Trend_{vname}', 'lb', 'ub']
    cmap = cm.get_cmap('PRGn_r', 11)
    titles =[f'{vname} Trend','Lower bound', 'Upper bound']
    for c1, c2, c3, c4,k, t in zip(col1,col2, col3, col4,keys, titles):

        ec_trend[k].plot(ax=ax[c1], transform = ccrs.PlateCarree(), vmin=-50, vmax=50, cmap=cmap, 
                         add_colorbar=False, add_labels=False)
        mpi_trend[k].plot(ax=ax[c2], transform = ccrs.PlateCarree(), vmin=-50, vmax=50, cmap=cmap, add_colorbar=False,
                         add_labels=False)
        nor_trend[k].plot(ax=ax[c3], transform = ccrs.PlateCarree(), vmin=-50, vmax=50, cmap=cmap, add_colorbar=False,
                         add_labels=False)
        rel_forces [k].plot(ax=ax[c4], transform = ccrs.PlateCarree(), vmin=-50, vmax=50, cmap=cmap, add_colorbar=False,
                         add_labels=False)
        ax[c1].set_title(t, fontsize=10)
        ax[c2].set_title(t,fontsize=10)
        ax[c3].set_title(t,fontsize=10)
        ax[c4].set_title(t,fontsize=10)
    for a in ax:
        ax[a].coastlines()

    fig.text(x=0.21,y=0.96,s='EC-Earth3-AerChem', ha='center', va='center', fontsize=16)
    fig.text(x=0.411,y=0.96,s='MPI-ESM-1-2-HAM', ha='center', va='center', fontsize=16)
    fig.text(x=0.616,y=0.96,s='NorESM2-LM', ha='center', va='center', fontsize=16)
    fig.text(x=0.817,y=0.96,s='FORCeS Combined', ha='center', va='center', fontsize=16)
    # ax['C'].set_title('NorESM2-LM')
    # ax['B'].set_title('MPI-ESM-1-2-HAM')
    # ax['D'].set_title('FORCeS combined')
    cb = fig.colorbar(cm.ScalarMappable(mpl.colors.Normalize(vmin=-50,vmax=50),cmap=cmap),cax=cax, extend='both')
    cb.set_label(f'Aerosol {vname} Trend % [year-1] %')
    plt.savefig(snakemake.output.plot_rel, dpi=144, bbox_inches='tight')
plot_relative_trends()

In [29]:
def plot_standardized_trends():
    nor_trend = sen_slope(norm_nor[vname])
    ec_trend = sen_slope(norm_ec[vname])
    mpi_trend = sen_slope(norm_mpi[vname])
    forces_standardized = sen_slope(merge_standardized)

    fig ,ax, cax = create_facet_plot(nplots=12, figsize=(15,6), subplot_kw={'projection':ccrs.Robinson()})
    col1 = ['A', 'E', 'I']
    col2 = ['B', 'F', 'L']
    col3 = ['C', 'G', 'M']
    col4 = ['D', 'H', 'N']
    keys = [f'Trend_{vname}', 'lb', 'ub']
    cmap = cm.get_cmap('PRGn_r', 11)
    titles =[f'Standardized {vname} Trend','Lower bound', 'Upper bound']
    for c1, c2, c3, c4,k, t in zip(col1,col2, col3, col4,keys, titles):

        ec_trend[k].plot(ax=ax[c1], transform = ccrs.PlateCarree(), vmin=-0.15, vmax=0.15, cmap=cmap, 
                         add_colorbar=False, add_labels=False)
        mpi_trend[k].plot(ax=ax[c2], transform = ccrs.PlateCarree(), vmin=-0.15, vmax=0.15, cmap=cmap, add_colorbar=False,
                         add_labels=False)
        nor_trend[k].plot(ax=ax[c3], transform = ccrs.PlateCarree(), vmin=-0.15, vmax=0.15, cmap=cmap, add_colorbar=False,
                         add_labels=False)
        forces_standardized[k].plot(ax=ax[c4], transform = ccrs.PlateCarree(), vmin=-0.15, vmax=0.15, cmap=cmap, add_colorbar=False,
                         add_labels=False)
        ax[c1].set_title(t, fontsize=10)
        ax[c2].set_title(t,fontsize=10)
        ax[c3].set_title(t,fontsize=10)
        ax[c4].set_title(t,fontsize=10)
    for a in ax:
        ax[a].coastlines()

    fig.text(x=0.21,y=0.96,s='EC-Earth3-AerChem', ha='center', va='center', fontsize=16)
    fig.text(x=0.411,y=0.96,s='MPI-ESM-1-2-HAM', ha='center', va='center', fontsize=16)
    fig.text(x=0.616,y=0.96,s='NorESM2-LM', ha='center', va='center', fontsize=16)
    fig.text(x=0.817,y=0.96,s='FORCeS Combined', ha='center', va='center', fontsize=16)
    # ax['C'].set_title('NorESM2-LM')
    # ax['B'].set_title('MPI-ESM-1-2-HAM')
    # ax['D'].set_title('FORCeS combined')
    cb = fig.colorbar(cm.ScalarMappable(mpl.colors.Normalize(vmin=-0.15,vmax=0.15),cmap=cmap),cax=cax, extend='both')
    cb.set_label(f'Standardized Trend {vname} [year-1]')
    plt.savefig(snakemake.output.plot_std, dpi=144, bbox_inches='tight')
plot_standardized_trends()

In [30]:
rel_forces = sen_slope(merge_rel)
rel_forces = rel_forces.cf.add_bounds(['lon','lat'])
# merged_rel = merge_rel.to_dataset(name='Merged_rel_ERF')
merged_rel = regrid_global(rel_forces,lat=5, lon=5)



In [32]:

mask = xr.where(np.abs(merged_rel[f'Trend_{vname}']) < 12,0,1)
# mask = xr.where(merged_rel['Trend_ERFt'] < 10,1,mask)

In [33]:
m = mask.to_dataset(name='ERF_region_mask')
m.attrs['treshold'] = mask_treshold
m.to_netcdf(snakemake.output.mask)

m['Asia'] = m['ERF_region_mask'].where((m.lon > 60) & ((m.lat > 20)&(m.lat < 50)),0.0)
m['North_America'] =m['ERF_region_mask'].where(((m.lon > -100)&(m.lon < -30)) & (m.lat > 20),0.0) 
m['Europe'] = m['ERF_region_mask'].where(((m.lon > 20)&(m.lon < 70)) & (m.lat > 20),0.0)
m.to_netcdf(snakemake.output.mask)